### Importing Required Libraries

In this section, we import the necessary Python libraries used throughout the notebook:

- **csv**: A built-in Python module for reading and writing CSV (Comma-Separated Values) files.
- **pandas**: A powerful data manipulation and analysis library, used here primarily to save data into Excel format.
- **openpyxl**: A library for reading and writing Excel 2010 xlsx/xlsm/xltx/xltm files. We use:
  - `load_workbook` to open existing Excel files,
  - `get_column_letter` to convert column numbers to Excel-style letters,
  - `Border`, `Side`, `PatternFill`, and `Font` for cell styling,
  - `ColorScaleRule` for conditional formatting.
- **matplotlib.pyplot**: A popular plotting library for creating static visualizations such as charts and graphs.

In [430]:
import csv # Import the built-in CSV module for reading CSV files
import pandas as pd  # Import pandas for saving to Excel
from openpyxl import load_workbook
from openpyxl.cell.cell import Cell
from openpyxl.utils import get_column_letter
from openpyxl.styles import Border, Side, PatternFill, Font, Alignment
from openpyxl.formatting.rule import ColorScaleRule

import matplotlib.pyplot as plt  # Import matplotlib for plotting

### Configuration Settings

This section defines configuration variables used to control the behavior of the script:

- **INCLUDE_SED**: A flag used to determine whether SED (Spectral Energy Distribution) columns should be included in the output.
  - Set to `1` to include SED columns.
  - Set to `0` to exclude them.
  
- **file_name**: The name of the input CSV file to be processed.
- **output_file**: The name of the Excel file where the processed data will be saved.

In [431]:
# Flag to control output format
INCLUDE_SED = 0  # Set to 1 to include SED columns, 0 to exclude them

# Define the CSV file name and output file name
file_name = "run_name_SED.csv"
output_file = "processed_data_SED.xlsx"

### Data Cleaning Function and Initialization

This part defines a utility function and initializes key data structures:

- **`clean_value(value)`**: A helper function for cleaning and converting values read from the CSV file.
  - Strips leading and trailing whitespace.
  - Converts numeric strings to either `int` or `float`, preserving the appropriate type.
  - Replaces empty or `None` values with `0`.
  - Returns the original string if conversion to a number is not possible.

- **`processed_data`**: A list to store cleaned and structured data from the CSV file.
- **`modes`**: A list to store mode information extracted during parsing (usage depends on later context).
- **`airfoil_ratios`**: A list to store airfoil ratio values (also populated later).

In [432]:
# Read the CSV file
def clean_value(value):
    """
    Cleans and converts CSV values:
    - Strips whitespace
    - Converts numerical values to int or float
    - Replaces empty values with 0
    """    
    if value is None or value.strip() == "": # Handle missing or empty values
        return 0
    value = value.strip() # Remove leading and trailing spaces
    
    try:
        num = float(value) # Attempt to convert to float
        if num.is_integer(): # If the number is an integer (e.g., 3.0 -> 3)
            return int(num)
        return num # Return as float if it has decimal places
    except ValueError:
        return value # If conversion fails, return the original string

# List to store processed data
processed_data = []
modes = []
airfoil_ratios = []


### Reading and Preparing the CSV File

This section opens the specified CSV file and prepares it for processing:

- The file is opened using `csv.DictReader`, which reads each row as a dictionary.
- Column names (fieldnames) are stripped of whitespace to ensure consistency.
- If column names are found, they are printed for verification. Otherwise, an error message is shown.

###  and ... Processing CSV Rows

Each row from the CSV is processed as follows:

- Values are cleaned using the `clean_value()` function.
- For each row, a dictionary `base` is created to store basic information:
  - `"Mode#"`: taken from the `mode` column.
  - `"Frequency [Hz]"`: taken from the `FREQ` column.
- Component ratios (`Airfoil`, `Inner Band`, `Honey Comb`, `Outer Band`) are extracted, multiplied by 100, and sorted in descending order.
- The top components and their percentages are added to the `base` dictionary.
- The result is appended to `processed_data`.

In [433]:
# Open the CSV file for reading	
with open(file_name, newline='') as csv_file:
    csv_reader = csv.DictReader(csv_file, delimiter=',') # Read CSV as dictionary

	# Strip spaces from column names to ensure consistency
    if csv_reader.fieldnames:
        csv_reader.fieldnames = [col.strip() for col in csv_reader.fieldnames]

    # Print column names if available, otherwise print an error message
    if csv_reader.fieldnames:
        print(f'Column names: {",".join(csv_reader.fieldnames)}')
    else:
        print('No column names found.')

# Process each row in the CSV file
    line_count = 0
    for row in csv_reader:
    # Clean each value in the row and store it in a new dictionary
        row = {key.strip(): clean_value(value) for key, value in row.items()}

    # Basic info
        base = {
            "Mode#": row["mode"],
            "Frequency [Hz]": row["FREQ"]
        }

        # Extract component ratios and sort
        components = {
            "Airfoil": row["SENE_AERO_ratio"],
            "Inner Band": row["SENE_IB_Ratio"],
            "Honey Comb": row["SENE_HC_Ratio"],
            "Outer Band": row["SENE_OB_Ratio"]
        }
        sorted_components = sorted(components.items(), key=lambda x: x[1], reverse=True)

        for i, (name, value) in enumerate(sorted_components, start=1):
            base[f"Component {i}"] = name
            base[f"%  {i}"] = value

        processed_data.append(base)
        line_count += 1

Column names: mode,FREQ,SENE_AERO_ratio,SENE_IB_Ratio,SENE_HC_Ratio,SENE_OB_Ratio,MAX_SED_LYNN,MAX_SED_ALT


### Converting to DataFrame and Exporting to Excel

This part of the code handles the final data export:

- The `processed_data` list is converted into a **pandas DataFrame** for easier handling and export.
- A message is printed showing the total number of processed rows and the output filename.
- The DataFrame is saved to an Excel file (`.xlsx`) using `to_excel()`, with `index=False` to exclude row indices.

In [434]:
# Convert to DataFrame and save to Excel
df = pd.DataFrame(processed_data)

# Print total processed lines
print(f'Processed {line_count} lines. Output saved to {output_file}')

# Create and export the sorted DataFrame
df_sorted = pd.DataFrame(processed_data)

df_sorted.to_excel(output_file, index=False)  # Save to Excel

Processed 43 lines. Output saved to processed_data_SED.xlsx


### Formatting Excel Output with openpyxl

To enhance the readability of the Excel file, the script reopens it using `openpyxl` for formatting:

- The first worksheet is accessed.
- A default font size of 14 is applied to all data cells (excluding the header row).

In [435]:
# Reopen with openpyxl to format cells
wb = load_workbook(output_file)
ws = wb[wb.sheetnames[0]]  # Get the first worksheet

ws.insert_cols(1)  # Insert a new column at the beginning
ws.insert_rows(1)  # Insert a new row at the top3

# Set default font for all cells
default_font = Font(size=18)
for row in ws.iter_rows(min_row=2, max_row=ws.max_row):
    for cell in row:
        cell.font = default_font

### Advanced Excel Formatting

To improve the visual appearance and readability of the Excel output, several formatting enhancements are applied using `openpyxl`:

1. **Auto-adjust Column Widths**  
   For each column, the maximum string length is calculated, and the width is adjusted accordingly to fit the content.

2. **Apply Thin Borders**  
   A thin border is applied to all cells in the worksheet, creating a clear table structure.

3. **Bold Header Row**  
   The first row (headers) is styled with a bold font to distinguish it from the rest of the data.

4. **Conditional ColorScale for Percentage Columns**  
   A color scale (red → yellow → green) is applied to the `%` columns (`%1`, `%2`, `%3`, `%4`) to visually represent value intensity.

5. **Static Fill Colors for % Columns**  
   Background fill colors are applied to the `%` columns for better visual segmentation:
   - `%1` → light orange
   - `%2` → light blue
   - `%3` → lavender
   - `%4` → light green

6. **Highlight 'Airfoil' Components**  
   If a component is labeled `"Airfoil"`, both its name and its associated percentage value are styled with bold red font to draw attention.

In [436]:
# 3. Set custom headers
new_headers = ["Mode", "Freq [Hz]"]
for i in range(1, 5):
    new_headers.append(f"Comp {i}")
    new_headers.append(f"Comp {i} [%]")

# 4 Apply new headers starting from column B, row 2
for col_idx, header in enumerate(new_headers, start=2):
    cell = ws.cell(row=2, column=col_idx)
    cell.value = header

# Set column widths based on header and data, accounting for inserted rows
for col_idx in range(2, ws.max_column + 1):  # Skip column A
    max_length = 0
    col_letter = get_column_letter(col_idx)

    # Check header in row 3 (after inserted row)
    header_cell = ws.cell(row=2, column=col_idx)
    if header_cell.value:
        max_length = len(str(header_cell.value))

    # Check all data rows
    for row_idx in range(3, ws.max_row + 1):
        cell = ws.cell(row=row_idx, column=col_idx)
        if cell.value:
            max_length = max(max_length, len(str(cell.value)))

    # Final width with generous padding
    adjusted_width = max(12, max_length * 1.5 + 4)
    ws.column_dimensions[col_letter].width = adjusted_width

    # Apply alignment settings to the whole column (header + data)
    for row_idx in range(2, ws.max_row + 1):
        cell = ws.cell(row=row_idx, column=col_idx)
        cell.alignment = Alignment(
            wrap_text=False,
            shrink_to_fit=False,
            horizontal='center')

# 2. Apply thin border to all cells
thin_border = Border(
    left=Side(style='thin'),
    right=Side(style='thin'),
    top=Side(style='thin'),
    bottom=Side(style='thin')
)

for row in ws.iter_rows(min_row=2, max_row=ws.max_row, min_col=2, max_col=ws.max_column):
    for cell in row:
        cell.border = thin_border

# 3. Apply double border blue line to all around the table
double_side = Side(style='double', color='19386B')

min_row, max_row = 2, ws.max_row
min_col, max_col = 2, ws.max_column

for row in range(min_row, max_row + 1):
    for col in range(min_col, max_col + 1):
        cell = ws.cell(row=row, column=col)
        
        # Create a border for the cell
        border = Border(
            left=double_side if col == min_col else None,
            right=double_side if col == max_col else None,
            top=double_side if row == min_row else None,
            bottom=double_side if row == max_row else None
        )

        # Apply the border to the cell
        cell.border = Border(
            left=border.left if border.left else cell.border.left,
            right=border.right if border.right else cell.border.right,
            top=border.top if border.top else cell.border.top,
            bottom=border.bottom if border.bottom else cell.border.bottom,
        )

# 4. Apply ColorScaleRule formatting to percentage columns
percent_columns = ['E', 'G', 'I', 'K']
color_scale_rule = ColorScaleRule(
    start_type='num', start_value=0.0, start_color='D9EAD3',
    mid_type='num', mid_value=0.5, mid_color='FCE5CD',
    end_type='num', end_value=1.0, end_color='E69138'
)

for col_letter in percent_columns:
    ws.conditional_formatting.add(f"{col_letter}3:{col_letter}{ws.max_row}", color_scale_rule)

# 5. Format percentage cells to show 'XX%' style
for col_letter in percent_columns:
    for row in range(3, ws.max_row + 1):
        cell = ws[f"{col_letter}{row}"]
        if isinstance(cell.value, (int, float)):
            cell.number_format = '0%'

# 6. RED FONT ONLY FOR 'Airfoil' with YELLOW FILL
highlight_font = Font(color='C00000', bold=True, size=18)  # Set font size
highlight_fill = PatternFill(fill_type='solid', fgColor='FFFE5D')

component_to_percent = ['D', 'F', 'H', 'J'] # List of component columns to check

for comp_col in component_to_percent:
    for row in range(3, ws.max_row + 1):
        comp_cell = ws[f"{comp_col}{row}"]
        if comp_cell.value == "Airfoil":
            comp_cell.font = highlight_font

            # Apply yellow fill to both cells (preserves existing number formatting)
            comp_cell.fill = highlight_fill

### Saving the Formatted Excel File

Finally, the modified Excel workbook is saved using `wb.save()` to persist all the applied formatting and styling changes.

In [437]:
# Save the formatted Excel file
wb.save(output_file)